In [1]:
import os

os.environ['OPENAI_API_KEY'] = '****'
os.environ['LANGCHAIN_API_KEY'] = '****'
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = '09-01'

In [2]:
!pip install -qU langchain_community langchain_openai langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.2/412.2 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 55.7 MB/s eta 0:00:00
 

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

#텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=0)

#텍스트 파일을 load -> List[Document] 형태로 변환
loader1 = TextLoader("/content/data/nlp-keywords.txt")
loader2 = TextLoader("/content/data/finance-keywords.txt")

#문서 분할
split_doc1 = loader1.load_and_split(text_splitter)
split_doc2 = loader2.load_and_split(text_splitter)

#문서 개수 확인
len(split_doc1), len(split_doc2)

(11, 6)

In [4]:
split_doc1[0]

Document(metadata={'source': '/content/data/nlp-keywords.txt'}, page_content='Semantic Search\n\n정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.\n예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.\n연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝\n\nEmbedding\n\n정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken\n\n정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.\n예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 분할합니다.\n연관키워드: 토큰화, 자연어 처리, 구문 분석\n\nTokenizer')

# VectorStore 생성

벡터 저장소 생성 (`from_documents`)  
`from_documents` 클래스 메서드는 문서 리스트로부터 벡터 저장소를 생성합니다.

## 매개변수

- **documents** (`List[Document]`): 벡터 저장소에 추가할 문서 리스트
- **embedding** (`Optional[Embeddings]`): 임베딩 함수. 기본값은 `None`
- **ids** (`Optional[List[str]]`): 문서 ID 리스트. 기본값은 `None`
- **collection_name** (`str`): 생성할 컬렉션 이름
- **persist_directory** (`Optional[str]`): 컬렉션을 저장할 디렉토리. 기본값은 `None`
- **client_settings** (`Optional[chromadb.config.Settings]`): Chroma 클라이언트 설정
- **client** (`Optional[chromadb.Client]`): Chroma 클라이언트 인스턴스
- **collection_metadata** (`Optional[Dict]`): 컬렉션 구성 정보. 기본값은 `None`


In [5]:
OpenAIEmbeddings(model='text-embedding-3-small')

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7e64ce1e67d0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7e65001f81d0>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

### 1. 저장

In [6]:
Chroma.from_documents(
    documents=split_doc1,
    embedding=OpenAIEmbeddings(model='text-embedding-3-small'),
    collection_name='my_db'
)


In [7]:
DB_PATH ="./chroma_db"

persist_db = Chroma.from_documents(
    split_doc1, OpenAIEmbeddings(), persist_directory=DB_PATH, collection_name='my_db'
)

### 2.불러오기

In [8]:
loadede_db = Chroma(
    persist_directory=DB_PATH, embedding_function=OpenAIEmbeddings() ,collection_name='my_db'
)

In [9]:
loadede_db.get()

{'ids': ['8421e4ae-5dfe-49d1-be38-08bd977bdab9',
  'e078cfa6-545b-4f79-a2ab-c087fce32520',
  '5b7f50df-3ad4-4fcb-b2fa-7d20bb606dd9',
  'f6130196-0862-4eba-b2f3-d90346b76172',
  '1c9e156f-86c3-4805-a499-2dcc55ec063b',
  '8cea42c0-fbe0-4e2e-bc66-21ef0a941911',
  'c6f40856-f514-405a-abc4-544e27c7bd84',
  'da71eb4a-c450-498c-a123-ab5a4e8a25d3',
  'a1274311-4718-430b-be3e-84c3f2673fc6',
  'e473d3cb-71fa-41fd-b32b-432ce24b72ed',
  '5b7786a5-91e1-41f2-a24a-8841fa69fe96'],
 'embeddings': None,
 'documents': ['Semantic Search\n\n정의: 의미론적 검색은 사용자의 질의를 단순한 키워드 매칭을 넘어서 그 의미를 파악하여 관련된 결과를 반환하는 검색 방식입니다.\n예시: 사용자가 "태양계 행성"이라고 검색하면, "목성", "화성" 등과 같이 관련된 행성에 대한 정보를 반환합니다.\n연관키워드: 자연어 처리, 검색 알고리즘, 데이터 마이닝\n\nEmbedding\n\n정의: 임베딩은 단어나 문장 같은 텍스트 데이터를 저차원의 연속적인 벡터로 변환하는 과정입니다. 이를 통해 컴퓨터가 텍스트를 이해하고 처리할 수 있게 합니다.\n예시: "사과"라는 단어를 [0.65, -0.23, 0.17]과 같은 벡터로 표현합니다.\n연관키워드: 자연어 처리, 벡터화, 딥러닝\n\nToken\n\n정의: 토큰은 텍스트를 더 작은 단위로 분할하는 것을 의미합니다. 이는 일반적으로 단어, 문장, 또는 구절일 수 있습니다.\n예시: 문장 "나는 학교에 간다"를 "나는", "학교에", "간다"로 

## Text DB 생성

In [10]:
db2 = Chroma.from_texts(
    ["안녕하세요. 정말 반갑습니다.", "제 이름은 손흥민 입니다."],
    embedding=OpenAIEmbeddings(),
)

In [11]:
db2.get()

{'ids': ['86e91359-00a6-4337-821f-2ae0029be783',
  '33c2d3d1-0ce6-4bb5-bd94-34493a8644fe'],
 'embeddings': None,
 'documents': ['안녕하세요. 정말 반갑습니다.', '제 이름은 손흥민 입니다.'],
 'uris': None,
 'data': None,
 'metadatas': [None, None],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

## 유사도 검색

In [12]:
db2.get()

{'ids': ['86e91359-00a6-4337-821f-2ae0029be783',
  '33c2d3d1-0ce6-4bb5-bd94-34493a8644fe'],
 'embeddings': None,
 'documents': ['안녕하세요. 정말 반갑습니다.', '제 이름은 손흥민 입니다.'],
 'uris': None,
 'data': None,
 'metadatas': [None, None],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [13]:
persist_db.similarity_search("TF IDF 에 대하여 알려줘")

[Document(id='c6f40856-f514-405a-abc4-544e27c7bd84', metadata={'source': '/content/data/nlp-keywords.txt'}, page_content='정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을 가집니다.\n연관키워드: 자연어 처리, 정보 검색, 데이터 마이닝\n\nDeep Learning\n\n정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다. 이는 데이터에서 고수준의 표현을 학습하는 데 중점을 둡니다.\n예시: 이미지 인식, 음성 인식, 자연어 처리 등에서 딥러닝 모델이 활용됩니다.\n연관키워드: 인공신경망, 머신러닝, 데이터 분석\n\nSchema\n\n정의: 스키마는 데이터베이스나 파일의 구조를 정의하는 것으로, 데이터가 어떻게 저장되고 조직되는지에 대한 청사진을 제공합니다.\n예시: 관계형 데이터베이스의 테이블 스키마는 열 이름, 데이터 타입, 키 제약 조건 등을 정의합니다.\n연관키워드: 데이터베이스, 데이터 모델링, 데이터 관리\n\nDataFrame'),
 Document(id='8cea42c0-fbe0-4e2e-bc66-21ef0a941911', metadata={'source': '/content/data/nlp-keywords.txt'}, page_content='정의: 오픈 소스는 소스 코드가 공개되어 누구나 자유롭게 사용, 수정, 배포할 수 있는 소프트웨어를 의미합니다. 이는 협업과 혁신을 촉진하는 데 중요한 역할을 합니다.\n예시: 리눅스 운영 체제는 대표적인 오픈 소스 프로젝트입니다.\n연관키워드: 소프트웨어 개발, 커뮤니티, 기술 협업\n\nStructured Data\n\n정의: 구조화된 데이터는 정해진 형식이나 스키마에 따라 조직

In [14]:
persist_db.similarity_search("TF IDF 에 대하여 알려줘",
                             k=1,
                             filter={"source":"/content/data/nlp-keywords.txt"}
                             )


[Document(id='c6f40856-f514-405a-abc4-544e27c7bd84', metadata={'source': '/content/data/nlp-keywords.txt'}, page_content='정의: TF-IDF는 문서 내에서 단어의 중요도를 평가하는 데 사용되는 통계적 척도입니다. 이는 문서 내 단어의 빈도와 전체 문서 집합에서 그 단어의 희소성을 고려합니다.\n예시: 많은 문서에서 자주 등장하지 않는 단어는 높은 TF-IDF 값을 가집니다.\n연관키워드: 자연어 처리, 정보 검색, 데이터 마이닝\n\nDeep Learning\n\n정의: 딥러닝은 인공신경망을 이용하여 복잡한 문제를 해결하는 머신러닝의 한 분야입니다. 이는 데이터에서 고수준의 표현을 학습하는 데 중점을 둡니다.\n예시: 이미지 인식, 음성 인식, 자연어 처리 등에서 딥러닝 모델이 활용됩니다.\n연관키워드: 인공신경망, 머신러닝, 데이터 분석\n\nSchema\n\n정의: 스키마는 데이터베이스나 파일의 구조를 정의하는 것으로, 데이터가 어떻게 저장되고 조직되는지에 대한 청사진을 제공합니다.\n예시: 관계형 데이터베이스의 테이블 스키마는 열 이름, 데이터 타입, 키 제약 조건 등을 정의합니다.\n연관키워드: 데이터베이스, 데이터 모델링, 데이터 관리\n\nDataFrame')]

### 도큐멘트 추가

In [15]:
from langchain_core.documents import Document

db2.add_documents(
    [
        Document(
            page_content = "나는 1번 문서 입니다.",
            metadata = {"source":"/content/data/mydata.txt"},
            id="1",
        )
    ]
)

['1']

In [16]:
db2.get()

{'ids': ['86e91359-00a6-4337-821f-2ae0029be783',
  '33c2d3d1-0ce6-4bb5-bd94-34493a8644fe',
  '1'],
 'embeddings': None,
 'documents': ['안녕하세요. 정말 반갑습니다.', '제 이름은 손흥민 입니다.', '나는 1번 문서 입니다.'],
 'uris': None,
 'data': None,
 'metadatas': [None, None, {'source': '/content/data/mydata.txt'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [17]:
#Document 추가

db2.add_texts(
    ["나는 2번 문서 입니다.", "나는 3번 문서 입니다."],
    metadatas=[{"source":"mydata.txt"}, {"source":" "}],
    ids=["2", "3"],
)

['2', '3']

In [18]:
db2.get()

{'ids': ['86e91359-00a6-4337-821f-2ae0029be783',
  '33c2d3d1-0ce6-4bb5-bd94-34493a8644fe',
  '1',
  '2',
  '3'],
 'embeddings': None,
 'documents': ['안녕하세요. 정말 반갑습니다.',
  '제 이름은 손흥민 입니다.',
  '나는 1번 문서 입니다.',
  '나는 2번 문서 입니다.',
  '나는 3번 문서 입니다.'],
 'uris': None,
 'data': None,
 'metadatas': [None,
  None,
  {'source': '/content/data/mydata.txt'},
  {'source': 'mydata.txt'},
  {'source': ' '}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

In [19]:
db2.get('1')

{'ids': ['1'],
 'embeddings': None,
 'documents': ['나는 1번 문서 입니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'source': '/content/data/mydata.txt'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

### 삭제

In [20]:
db2.delete(ids=["1"])

In [21]:
db2.get('1')

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

## 초기화(Collection init)

- 매우주의!!!

In [22]:
db2.reset_collection()

In [23]:
db2.get()

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

## 검색기(Retriever)
- `as_retriever` 메서드는 벡터 저장소를 기반으로 `VectorStoreRetriever`를 생성
-----------
### 매개변수

- **`**kwargs`**: 검색 함수에 전달할 키워드 인자
- **`search_type`** (Optional[str]): 검색 유형 ("similarity", "mmr", "similarity_score_threshold")
- **`search_kwargs`** (Optional[Dict]): 검색 함수에 전달할 추가 인자
- **`k`**: 반환할 문서 수 (기본값: 4)
- **`score_threshold`**: 최소 유사도 임계값
- **`fetch_k`**: MMR 알고리즘에 전달할 문서 수 (기본값: 20)
- **`lambda_mult`**: MMR 결과의 다양성 조절 (0~1, 기본값: 0.5)
- **`filter`**: 문서 메타데이터 필터링

### 반환값

- **`VectorStoreRetriever`**: 벡터 저장소 기반 검색기 인스턴스를 생성합니다.


In [24]:
db = Chroma.from_documents(
    documents=split_doc1 + split_doc2,
    embedding=OpenAIEmbeddings(),
    collection_name='nlp',
)

In [30]:
retriever = db.as_retriever()
retriever.invoke('HuggingFace가 뭔가요?')

[Document(id='f5f6d487-584a-407a-9801-0853c983074f', metadata={'source': '/content/data/nlp-keywords.txt'}, page_content='정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니다.\n예시: HuggingFace의 Transformers 라이브러리를 사용하여 감정 분석, 텍스트 생성 등의 작업을 수행할 수 있습니다.\n연관키워드: 자연어 처리, 딥러닝, 라이브러리\n\nDigital Transformation\n\n정의: 디지털 변환은 기술을 활용하여 기업의 서비스, 문화, 운영을 혁신하는 과정입니다. 이는 비즈니스 모델을 개선하고 디지털 기술을 통해 경쟁력을 높이는 데 중점을 둡니다.\n예시: 기업이 클라우드 컴퓨팅을 도입하여 데이터 저장과 처리를 혁신하는 것은 디지털 변환의 예입니다.\n연관키워드: 혁신, 기술, 비즈니스 모델\n\nCrawling\n\n정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.\n예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.\n연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진\n\nWord2Vec'),
 Document(id='2b83f889-8f27-47dc-9912-278a046cebc5', metadata={'source': '/content/data/nlp-keywords.txt'}, page_content='정의: CSV(Comma-Separated Values)는 데이터를 저장하는 파일 형식으로, 각 데이터 값은 쉼표로 구분됩니다. 표 형태의 데이터를 간단하게 저장하고 교환할 때 사용됩니다.\n예시: 이름, 나이, 직업이라는 헤더를 가진 CSV 파일에는 

In [36]:
# mmr은 키워드로 유사도 검사하는 것임

retriever = db.as_retriever(
    search_type="mmr", search_kwargs={"k": 1, "lambda_mult": 0.25, "fetch_k": 10}
)

retriever.invoke("Huggingface 에 대하여 알려줘")

[Document(id='f5f6d487-584a-407a-9801-0853c983074f', metadata={'source': '/content/data/nlp-keywords.txt'}, page_content='정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니다.\n예시: HuggingFace의 Transformers 라이브러리를 사용하여 감정 분석, 텍스트 생성 등의 작업을 수행할 수 있습니다.\n연관키워드: 자연어 처리, 딥러닝, 라이브러리\n\nDigital Transformation\n\n정의: 디지털 변환은 기술을 활용하여 기업의 서비스, 문화, 운영을 혁신하는 과정입니다. 이는 비즈니스 모델을 개선하고 디지털 기술을 통해 경쟁력을 높이는 데 중점을 둡니다.\n예시: 기업이 클라우드 컴퓨팅을 도입하여 데이터 저장과 처리를 혁신하는 것은 디지털 변환의 예입니다.\n연관키워드: 혁신, 기술, 비즈니스 모델\n\nCrawling\n\n정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.\n예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.\n연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진\n\nWord2Vec')]

## 유사도 Threshold

In [45]:
retriever = db.as_retriever(
    search_type = "similarity_score_threshold", search_kwargs={"score_threshold": 0.8}
)

response = retriever.invoke("Word2Vec 에 대하여 알려줘")

In [46]:
len(response)

1

In [50]:
retriever = db.as_retriever(search_kwargs={'k':1})

retriever.invoke("Word2Vec 에 대하여 알려줘")

[Document(id='3c245422-faed-49f7-b782-0fef5270e5d2', metadata={'source': '/content/data/nlp-keywords.txt'}, page_content='정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이해 및 생성 작업에 사용됩니다.\n예시: OpenAI의 GPT 시리즈는 대표적인 대규모 언어 모델입니다.\n연관키워드: 자연어 처리, 딥러닝, 텍스트 생성\n\nFAISS (Facebook AI Similarity Search)\n\n정의: FAISS는 페이스북에서 개발한 고속 유사성 검색 라이브러리로, 특히 대규모 벡터 집합에서 유사 벡터를 효과적으로 검색할 수 있도록 설계되었습니다.\n예시: 수백만 개의 이미지 벡터 중에서 비슷한 이미지를 빠르게 찾는 데 FAISS가 사용될 수 있습니다.\n연관키워드: 벡터 검색, 머신러닝, 데이터베이스 최적화\n\nOpen Source')]

In [52]:
retriever = db.as_retriever(
    search_kwargs={"filter": {"source": "/content/data/finance-keywords.txt"}, "k": 2}
)
retriever.invoke("ESG 에 대하여 알려줘")


[Document(id='9bea7125-ade7-43e6-9123-9c6a28bb6cf8', metadata={'source': '/content/data/finance-keywords.txt'}, page_content='정의: ESG는 기업의 환경, 사회, 지배구조 측면을 고려하는 투자 접근 방식입니다.\n예시: S&P 500 ESG 지수는 우수한 ESG 성과를 보이는 기업들로 구성된 지수입니다.\n연관키워드: 지속가능 투자, 기업의 사회적 책임, 윤리 경영\n\nStock Buyback\n\n정의: 자사주 매입은 기업이 자사의 주식을 시장에서 다시 사들이는 것을 말합니다.\n예시: 애플은 S&P 500 기업 중 가장 큰 규모의 자사주 매입 프로그램을 운영하고 있습니다.\n연관키워드: 주주 가치, 자본 관리, 주가 부양\n\nCyclical Stocks\n\n정의: 경기순환주는 경제 상황에 따라 실적이 크게 변동하는 기업의 주식을 말합니다.\n예시: 포드, 제너럴 모터스와 같은 자동차 기업들은 S&P 500에 포함된 대표적인 경기순환주입니다.\n연관키워드: 경제 사이클, 섹터 분석, 투자 타이밍\n\nDefensive Stocks\n\n정의: 방어주는 경기 변동에 상관없이 안정적인 실적을 보이는 기업의 주식을 의미합니다.\n예시: 프록터앤갬블, 존슨앤존슨과 같은 생활필수품 기업들은 S&P 500 내 대표적인 방어주로 꼽힙니다.\n연관키워드: 안정적 수익, 저변동성, 리스크 관리'),
 Document(id='a79769b2-7749-4359-aeec-ffd16062b09f', metadata={'source': '/content/data/finance-keywords.txt'}, page_content='정의: 주식 리서치는 기업의 재무 상태, 사업 모델, 경쟁력 등을 분석하여 투자 의사 결정을 돕는 활동입니다.\n예시: 골드만삭스의 애널리스트들이 S&P 500 기업들에 대한 분기별 실적 전망을 발표했습니다.\n연관키워드: 투자 분석, 기업 가치평가, 시